In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


# ----------------------
# Load Data
# ----------------------
train_df = pd.read_csv('/kaggle/input/catch-me-if-you-can-alice-no-original/train_sessions.csv')
test_df = pd.read_csv('/kaggle/input/catch-me-if-you-can-alice-no-original/test_sessions.csv')
y_train = train_df['target']
train_df = train_df.drop(columns=['target'])
test_df['target'] = -1
df = pd.concat([train_df, test_df], ignore_index=True)

# Load site dictionary (optional)
with open('/kaggle/input/catch-me-if-you-can-alice-no-original/site_dic.pkl', 'rb') as f:
    site_dic = pickle.load(f)


In [3]:
# Fill NaNs
df.fillna(0, inplace=True)

# Time columns
time_cols = [col for col in df.columns if col.startswith('time')]
df[time_cols] = df[time_cols].apply(pd.to_datetime, errors='coerce')

# Add time differences between consecutive visits
def add_time_diffs_at_seconds(df):
    time_cols = sorted([col for col in df.columns if col.startswith('time')])
    df[time_cols] = df[time_cols].apply(pd.to_datetime, errors='coerce')
    
    for i in range(len(time_cols)-1):
        col1 = time_cols[i]
        col2 = time_cols[i+1] 
        diff_col = f'diff_{col1}_{col2}'
        df[diff_col] = (df[col2] - df[col1]).dt.total_seconds()
        
    return df

df = add_time_diffs_at_seconds(df)


In [4]:
df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,target,diff_time1_time10,diff_time10_time2,diff_time2_time3,diff_time3_time4,diff_time4_time5,diff_time5_time6,diff_time6_time7,diff_time7_time8,diff_time8_time9
0,1,718,2014-02-20 10:02:45,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,...,0.0,-1.392891e+09,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,...,0.0,2.600000e+01,-2.600000e+01,1.0,0.000000e+00,0.0,0.0,1.0,0.0,23.0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,...,0.0,7.000000e+00,-6.000000e+00,1.0,0.000000e+00,0.0,0.0,1.0,1.0,1.0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,...,0.0,2.700000e+02,-2.400000e+02,30.0,3.000000e+01,30.0,30.0,30.0,30.0,30.0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,...,0.0,2.460000e+02,-1.090000e+02,0.0,1.000000e+00,0.0,36.0,0.0,0.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,82793,812,2014-10-02 18:20:09,1039.0,2014-10-02 18:20:09,676.0,2014-10-02 18:20:09,0.0,1970-01-01 00:00:00,0.0,...,-1.0,-1.412274e+09,1.412274e+09,0.0,-1.412274e+09,0.0,0.0,0.0,0.0,0.0
336354,82794,300,2014-05-26 14:16:40,302.0,2014-05-26 14:16:41,302.0,2014-05-26 14:16:44,300.0,2014-05-26 14:16:44,300.0,...,-1.0,3.900000e+01,-3.800000e+01,3.0,0.000000e+00,35.0,0.0,0.0,0.0,0.0
336355,82795,29,2014-05-02 11:21:56,33.0,2014-05-02 11:21:56,35.0,2014-05-02 11:21:56,22.0,2014-05-02 11:22:03,37.0,...,-1.0,8.000000e+00,-8.000000e+00,0.0,7.000000e+00,0.0,0.0,0.0,1.0,0.0
336356,82796,5828,2014-05-03 10:05:25,23.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:27,804.0,2014-05-03 10:05:27,21.0,...,-1.0,1.300000e+01,-1.100000e+01,0.0,0.000000e+00,9.0,1.0,0.0,1.0,0.0


In [5]:
# Extract time-based features
def time_features(df):
    time_cols = sorted([col for col in df.columns if col.startswith('time')])
    df[time_cols] = df[time_cols].apply(pd.to_datetime, errors='coerce')
    
    for col in time_cols:
        df[f'year_{col}'] = df[col].dt.year
        df[f'month_{col}'] = df[col].dt.month
        df[f'day_{col}'] = df[col].dt.day
        df[f'weekday_{col}'] = df[col].dt.dayofweek
        df[f'hour_{col}'] = df[col].dt.hour
        df[f'minute_{col}'] = df[col].dt.minute
        df[f'second_{col}'] = df[col].dt.second
        df.drop(columns=col, inplace=True)  # Drop original datetime column
        
    return df

df = time_features(df)

In [6]:
# Cap large time diff outliers
def cap_large_values(df, threshold=86400):  # 1 day in seconds
    time_diff_cols = [col for col in df.columns if col.startswith('diff_')]
    for col in time_diff_cols:
        mask = df[col].abs() > threshold
        df.loc[mask, col] = threshold * df.loc[mask, col].apply(lambda x: 1 if x > 0 else -1)
    return df

df = cap_large_values(df)

In [7]:
# Session-level features
site_cols = [col for col in df.columns if col.startswith('site')]
df['session_length'] = df[site_cols].notnull().sum(axis=1)
df['unique_sites_count'] = df[site_cols].nunique(axis=1)

In [8]:
df['site_sequence'] = df[site_cols].fillna(0).astype(int).astype(str).agg(' '.join, axis=1)

vectorizer = TfidfVectorizer(max_features=2500)
tfidf_matrix = vectorizer.fit_transform(df['site_sequence'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f'tfidf_{i}' for i in range(tfidf_matrix.shape[1])])


In [9]:
# Final Cleanup
df = df.drop(columns=site_cols + ['site_sequence'])

In [10]:
# Concatenate TF-IDF features
df_final = pd.concat([df.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

In [11]:
df_final

,session_id,target,diff_time1_time10,diff_time10_time2,diff_time2_time3,diff_time3_time4,diff_time4_time5,diff_time5_time6,diff_time6_time7,diff_time7_time8,...,tfidf_2490,tfidf_2491,tfidf_2492,tfidf_2493,tfidf_2494,tfidf_2495,tfidf_2496,tfidf_2497,tfidf_2498,tfidf_2499
0,1,0.0,-86400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,26.0,-26.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,7.0,-6.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,270.0,-240.0,30.0,30.0,30.0,30.0,30.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,246.0,-109.0,0.0,1.0,0.0,36.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,82793,-1.0,-86400.0,86400.0,0.0,-86400.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336354,82794,-1.0,39.0,-38.0,3.0,0.0,35.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336355,82795,-1.0,8.0,-8.0,0.0,7.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336356,82796,-1.0,13.0,-11.0,0.0,0.0,9.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Split train/test
X_train = df_final.iloc[:len(y_train)].copy()
X_test = df_final.iloc[len(y_train):].copy()

# Attach labels
X_train['target'] = y_train

print("✅ Preprocessing complete")
print("Final train shape:", X_train.shape)
print("Final test shape:", X_test.shape)

✅ Preprocessing complete
Final train shape: (253561, 2583)
Final test shape: (82797, 2583)


In [13]:
X_train.head()

,session_id,target,diff_time1_time10,diff_time10_time2,diff_time2_time3,diff_time3_time4,diff_time4_time5,diff_time5_time6,diff_time6_time7,diff_time7_time8,...,tfidf_2490,tfidf_2491,tfidf_2492,tfidf_2493,tfidf_2494,tfidf_2495,tfidf_2496,tfidf_2497,tfidf_2498,tfidf_2499
0,1,0,-86400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,26.0,-26.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,7.0,-6.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,270.0,-240.0,30.0,30.0,30.0,30.0,30.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,246.0,-109.0,0.0,1.0,0.0,36.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X_test.head()

,session_id,target,diff_time1_time10,diff_time10_time2,diff_time2_time3,diff_time3_time4,diff_time4_time5,diff_time5_time6,diff_time6_time7,diff_time7_time8,...,tfidf_2490,tfidf_2491,tfidf_2492,tfidf_2493,tfidf_2494,tfidf_2495,tfidf_2496,tfidf_2497,tfidf_2498,tfidf_2499
253561,1,-1.0,7.0,-7.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253562,2,-1.0,85.0,-60.0,5.0,8.0,3.0,1.0,13.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253563,3,-1.0,84.0,-83.0,1.0,61.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253564,4,-1.0,4.0,-4.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253565,5,-1.0,13.0,-12.0,1.0,6.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253561 entries, 0 to 253560
Columns: 2583 entries, session_id to tfidf_2499
dtypes: float64(2509), int32(70), int64(4)
memory usage: 4.8 GB


In [16]:
X_train.shape

(253561, 2583)

In [17]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82797 entries, 253561 to 336357
Columns: 2583 entries, session_id to tfidf_2499
dtypes: float64(2510), int32(70), int64(3)
memory usage: 1.6 GB


In [18]:
X_test.shape

(82797, 2583)

In [19]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

2025-06-02 10:02:17.651288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748858537.928849     295 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748858538.009901     295 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available: 2


In [20]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))

GPU Available: True
GPU Name: Tesla T4


**LogisticRegression**

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

from cuml.linear_model import LogisticRegression as cuLogisticRegression
import cuml

In [19]:
# Separate features and target
X = X_train.drop(columns=['target'])
y = X_train['target']

# Split the training data
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = cuLogisticRegression(penalty='l1', max_iter=100, solver='qn')
model.fit(X_tr.astype('float32'), y_tr.astype('int32'))

LogisticRegression()

In [20]:
# Get probability predictions
probas = model.predict_proba(X_val)

# If it's a DataFrame or something else, convert it to NumPy
if hasattr(probas, "to_numpy"):
    probas = probas.to_numpy()

# Extract probability of class 1 (Alice)
y_pred_proba = probas[:, 1]

# Get predicted class labels
y_pred = model.predict(X_val)
if hasattr(y_pred, "to_numpy"):
    y_pred = y_pred.to_numpy()


In [21]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print("\nClassification Report:\n", classification_report(y_val, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))


Accuracy: 0.9909
ROC AUC Score: 0.5056

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     50254
           1       0.00      0.00      0.00       459

    accuracy                           0.99     50713
   macro avg       0.50      0.50      0.50     50713
weighted avg       0.98      0.99      0.99     50713


Confusion Matrix:
 [[50254     0]
 [  459     0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**XGBoost**

In [21]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

In [22]:
# Prepare features and labels
X = X_train.drop(columns=['target'])
y = X_train['target']

X = X.astype('float32')
y = y.astype('int32')

# Split data
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [23]:
# Initialize and train XGBoost model using GPU
model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    tree_method='gpu_hist',         # Enable GPU acceleration
    predictor='gpu_predictor'       # Use GPU for prediction as well
)

model.fit(X_tr, y_tr)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [10:03:09] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [10:03:09] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, predictor='gpu_predictor', ...)

In [24]:
# Predict
y_pred_proba = model.predict_proba(X_val)[:, 1]
y_pred = model.predict(X_val)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [10:03:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [10:03:52] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [25]:
# Evaluation
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print("\nClassification Report:\n", classification_report(y_val, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))

Accuracy: 0.9955
ROC AUC Score: 0.9918

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     50254
           1       0.95      0.53      0.68       459

    accuracy                           1.00     50713
   macro avg       0.97      0.77      0.84     50713
weighted avg       1.00      1.00      0.99     50713


Confusion Matrix:
 [[50241    13]
 [  215   244]]
